# Hidden Markov Model Implementation

#### We essentially will create a model with calculated weights and use that with the Viterbi algorithm to determine the accuracy of a given piano practice.

In [11]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# We define our model states using MIDI for labeling notes
ode15 = [64, 64, 65, 67, 67, 65, 64, 62, 60, 60, 62, 64, 64, 62, 62]
beats = [1]*15
events = list(zip(ode15, dur_beats))

NOTE_NAMES = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]


### Utility functions

##### We are using the build_hmm_from_beats() method below to essentially create a left-to-right hidden markov model with pre-defined weights that are based upon the CQT fram paramenters.

My CQT implementation uses 2 main parameters: hop length and sample rate. There is one spectogram column for every 1 hop length samples. So, the time between two columns is:

$dt = \frac{hoplen}{sr}$ seconds, or $1000 * \frac{hoplen}{sr}$ milliseconds

Secondly, we define a reference/ideal tempo, which in this case is bpm_ref = 120 beats per minute. When we convert this to milliseconds, this means 1 beat = 500 milliseconds. We can then calculate frames per beat to be (ms per beat) / (dt).



In [16]:
def mp3_to_cqt(audio_path, sr=22050, hop_length=512, fmin=27.5, n_bins=84, bins_per_octave=12):
    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    y, _ = librosa.effects.trim(y, top_db=30)

    C = librosa.cqt(y, sr=sr, hop_length=hop_length, fmin=fmin, n_bins=n_bins, bins_per_octave=bins_per_octave)
    C_mag = np.abs(C)
    C_db = librosa.amplitude_to_db(C_mag, ref=np.max)
    C_norm = (C_db - C_db.min()) / (C_db.max() - C_db.min() + 1e-8)

    return C_norm


def dt_ms_from_cqt(sr, hop_length):
    return 1000.0 * hop_length / sr


def build_hmm_from_beats(pitches, beats, bpm_ref, dt_ms):
    assert len(pitches) == len(beats)
    ms_per_beat = 60000.0 / bpm_ref
    frames_per_beat = ms_per_beat / dt_ms

    states = []
    for idx, (pitch, dur) in enumerate(zip(pitches, beats)):
        dur_steps = max(1, int(round(dur * frames_per_beat)))
        states.append({"idx": idx, "pitch": pitch, "beats": dur, "dur_steps": dur_steps})

    N = len(states)
    A = np.zeros((N, N), float)

    for i, s in enumerate(states):
        if i == N - 1:
            A[i, i] = 1.0
        else:
            p_adv = 1.0 / s["dur_steps"]
            A[i, i] = 1.0 - p_adv
            A[i, i+1] = p_adv

    pi = np.zeros(N); pi[0] = 1.0
    return states, A, pi


def midi_to_name(m):
    return f"{NOTE_NAMES[m % 12]}{(m // 12) - 1}"



In [17]:
def print_hmm_model(states, A, dt_ms):
    print("=== HMM Model ===")
    print(f"Number of states: {len(states)}")
    print()

    for i, s in enumerate(states):
        note = midi_to_name(s["pitch"])
        dur_frames = s["dur_steps"]
        dur_ms = dur_frames * dt_ms
        
        stay_prob = A[i, i]
        adv_prob = A[i, i+1] if i < len(states)-1 else 0.0

        print(f"State {i}: {note}")
        print(f"  Expected duration: {dur_frames} frames (~{dur_ms:.1f} ms)")
        print(f"  Transition: stay={stay_prob:.4f}, advance={adv_prob:.4f}")
        print()

In [19]:
bpm_ref = 120
sr = 22050
hop_length = 512
dt_ms = dt_ms_from_cqt(sr, hop_length)

states, A, pi = build_hmm_from_beats(ode15, beats, bpm_ref, dt_ms)
print_hmm_model(states, A, dt_ms)

=== HMM Model ===
Number of states: 15

State 0: E4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 1: E4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 2: F4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 3: G4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 4: G4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 5: F4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 6: E4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 7: D4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 8: C4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.9545, advance=0.0455

State 9: C4
  Expected duration: 22 frames (~510.8 ms)
  Transition: stay=0.95